<a href="https://colab.research.google.com/github/fhac-ewi/recurrent-neural-network/blob/main/%C3%9Cbung_RNN_L%C3%B6sung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Übung 7 - Rekurrente neuronale Netze (RNNs)

## In dieser Übung ...
... beschäftigen wir uns mit rekurrenten neuronalen Netzen, die unter anderem für die Textvorhersage (`text prediction`) eingesetzt werden können. Sie werden ein eigenes Modell zur Vorhersage weiterer Buchstaben (Zeichen) zu einer gegebenen Buchstabensequenz erstellen.

Dazu werden Sie die drei aus der Vorlesung bekannten Varianten von RNNs (Standard, LSTM und GRU) mit Keras implementieren und die Genauigkeit (Accuracy) der Netze miteinander vergleichen.


## 7.0 Vorbereitung

In diesem Abschnitt müssen Sie nichts programmieren! 🎉

Wir haben bereits alle notwendigen Imports für diese Übung hinzugefügt, sodass Sie direkt starten können. Sie müssen lediglich die GPU Unterstützung in Google Colab aktivieren.

**Ihre Aufgaben**

(1) Aktivieren Sie bitte die GPU Unterstützung in Google Colab. Wechseln Sie unter dem Reiter `Laufzeit` -> `Laufzeittyp ändern` von `None` auf `GPU` und bestätigen Sie diese Änderung.

(2) Führen Sie den folgenden Codezelle aus, um die erforderlichen Bibliotheken zu importieren und die GPU Unterstützung zu prüfen. 

  * Hinweis: Wenn Sie dieses Notebook mit dem Direktlink von GitHub geöffnet haben, wird bei der erstmaligen Ausführung eine Warnung angezeigt. Diese müssen Sie durch den Klick auf `Trotzdem ausführen` bestätigen. 

In [1]:
# Import everything needed for this exercise 
import tensorflow as tf
import keras
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from termcolor import colored

print(f"Keras Version: {keras.__version__}; Tensorflow version: {tf.__version__}; NumPy version: {np.__version__}; Python version: ", ".".join(str(x) for x in sys.version_info[:3]))

# Reset random number generators
seed = 1337
np.random.seed(seed)
tf.random.set_seed(seed)

# Check GPU support
from tensorflow.python.client import device_lib
physical_devices = device_lib.list_local_devices()
print("You are using", len(physical_devices), "local devises.", len([x for x in physical_devices if x.device_type == "GPU"]), "are GPUs")
for i, d in enumerate(physical_devices):
    print("  ->Device", i+1, "is a", d.device_type, "=>", d.physical_device_desc if len(d.physical_device_desc) > 0 else d.name)

if len([x for x in physical_devices if x.device_type == "GPU"]) == 0:
  raise Exception("Please enable GPU support before using this notebook. See here: [Runtime] -> [Change runtime type]")    

print("\n🎉🎉🎉 You are ready to go! 🎉🎉🎉")  

Keras Version: 2.4.3; Tensorflow version: 2.4.1; NumPy version: 1.19.5; Python version:  3.7.10
You are using 2 local devises. 1 are GPUs
  ->Device 1 is a CPU => /device:CPU:0
  ->Device 2 is a GPU => device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

🎉🎉🎉 You are ready to go! 🎉🎉🎉


## 7.1 One Hot Kodierung

Wir haben in der Vorlesung die One Hot Kodierung (1-aus-n-Code) kennen gelernt. Diese ermöglicht es Zeichen eines Alphabets als Vektoren darzustellen. 

Für ein Alphabet mit `n` einzigartigen Zeichen wird ein Vektor der Länge `n` benötigt. Jedes einzigartige Zeichen wird hierbei als Einheitsvektor definiert. Eine Zeichensequenz mit `m` Zeichen werden als `m` Vektoren mit jeweils der Länge `n` dargestellt.  

**Hinweis** Die vorliegende Implementierung ist für den Umgang mit Zeichen außerhalb des Alphabets konzipiert. Deshalb wird zur Darstellung eines Alphabets mit `n` einzigartigen Zeichen ein Vektor der Länge `n+1` erstellt. Alle Zeichen außerhalb des Alphabets werden durch das `unknown_token` ersetzt. 

Für das Training unseres Netzes werden wir die One Hot Kodierung zur Umwandlung eines Textes in Trainingsdaten nutzen. (Aufgabe 7.2)

* Hinweis: Für weitere Informationen zur One Hot Kodierun schlagen Sie diese in der Vorlesung nach oder benutzen Sie das [Internet](https://de.wikipedia.org/wiki/1-aus-n-Code).

In [2]:
class OneHot(object):
    def __init__(self, tokens, unknown_token = "[UNKNOWN]"):
        '''
        tokens: The tokens you want to be able to encode and decode
        unknown_token: The token to be used when decoding and the net wants to use a not known char
        '''
        self.tokens = tokens
        self.unknown_token = unknown_token
        # Store a bidirectional dictionary containing the characters
        self.char_to_index = dict((token, i + 1) for i, token in enumerate(self.tokens))
        self.index_to_char = dict((i + 1, token) for i, token in enumerate(self.tokens))
        pass

    def encode(self, text_as_tokens, dtype=np.bool_):
        '''
        text_as_tokens: List of tokens
        '''
        # Create the encoding matrix
        enc = np.empty((len(text_as_tokens), len(self.tokens) + 1), dtype=dtype)
        for i, token in enumerate(text_as_tokens):
            # Encode every char
            enc[i] = self.encode_token(token=token, l=len(self.tokens) + 1, dtype=dtype)
        return enc
    
    def encode_token(self, token, l, dtype=np.bool_):
        '''
        token: Single token
        l: length of array to be returned. At position zero it is stored if the char is unknown
        '''
        ret = np.zeros((1, l), dtype=dtype)
        if token not in self.char_to_index:
            ret[0, 0] = 1
        else:
            ret[0, self.char_to_index[token]] = 1
        return ret
    
    def decode(self, mat, unknown_token=None):         
        '''
        mat: matrix to be decoded. Has to be of shape (len_of_text, vocab_size)
        unknown_token: Unknown token. If none uses the one from __init__
        '''
        return [self.decode_token(mat[x]) for x in range(mat.shape[0])]
    
    def decode_token(self, vec, unknown_token=None):
        '''
        vec: Vector that should be decoded. Has to be on vocab length
        unknown_token: Unknown token. If none uses the one from __init__
        '''
        if unknown_token is None:
            unknown_token = self.unknown_token
        if isinstance(vec, tf.Tensor):
            vec = vec.numpy()
        if isinstance(vec, np.ndarray):
            # Use argmax since this will be used in the model created later
            am = np.argmax(vec)
        else:
            am = vec
        if am == 0:
            return unknown_token
        return self.index_to_char[am]

**Ihre Aufgaben**

(1) Untersuchen Sie gegebene Klasse zur One Hot Kodierung und die Funktion der einzelnen Methoden. 

(2) Erstellen Sie für das folgendes Alphabet `HWedlor` eine Instanz der Klasse `OneHot`. 

(3) Kodieren Sie nun die Zeichenfolge `Hello World!` mit der One Hot Kodierung. Wie wird das Zeichen `H` dargestellt? Geben Sie die Dimensionen des Vektors an.

* Hinweis: Sie können zur Umwandlung des Strings in eine Liste von Zeichen die Funktion `tokenize` verwenden. Dies ist aber nicht zwingend erforderlich, da Python den String automatisch als Liste interpretieren kann.

> <Antwort hier einfügen>

> **Musterlösung:** Das Zeichen `H` wird als Vektor `[False  True False False False False False False]` 
dargestellt. Die Dimension ist `1x8`.

(4) Wandeln Sie die kodierte Zeichenfolge zurück in einen für Menschen lesbaren Text. Entspricht der zurückgewandelte Text dem ursprünglichen Input? Falls nicht, beschreiben Sie die Ursache.

* Hinweis: Für eine schönere Ausgabe kann der enkodierte Text mit der Funktion `untokenize` in einen String gewandelt werden.

> <Antwort hier einfügen>

> **Musterlösung:** Der ursprüngliche Text und der kodierte und dekodierte Text ist nicht mehr identisch, da zwei Zeichen (Leerzeichen, Ausrufezeichen) nicht im Alphabet vorhanden sind. Durch Hinzufügen dieser Zeichen ins Alphabet würde man den ursprünglichen Text erhalten.

In [3]:
def tokenize(text):
    """
    Converts the given text (string) in a list of chars.
    """
    return list(text)

def untokenize(tokens):
    """
    Converts the given tokens (list of chars) in a string.
    """
    return "".join(tokens)

tokens = 'HWedlor'

# code here
one_hot = OneHot(tokens=tokens)

text='Hello World!'

coded_text = one_hot.encode(text)
print("Shape of coded text: ", coded_text.shape)
print("H coded (first letter in coded text): ", coded_text[0])

encoded_text = one_hot.decode(coded_text)

print(untokenize(encoded_text))

Shape of coded text:  (12, 8)
H coded (first letter in coded text):  [False  True False False False False False False]
Hello[UNKNOWN]World[UNKNOWN]


## 7.2 Datensatz vorbereiten

In dieser Übung verwenden wir das Buch Shakespeare TODO als Datensatz zum Training und Test unserer Modelle. Ziel der Modelle ist die Vorhersage weiterer Zeichen zu einer gegebenen Zeichensequenz.

In diesem Aufgabenteil werden die Daten eingelesen, mithilfe der One Hot Kodierung umgewandelt und anschließend in Trainingsdaten und Validierungsdaten aufgeteilt.

* Hinweis: Sie können theoretisch jeden beliebigen Text als Datensatz für das Training des Modells verwenden. (z.B. Harry Potter Bücher, Coronaschutzverordnungen, Nachrichtenartikel, ...)

**Ihre Aufgaben**

(1) Führen Sie die folgende Codezelle aus, um das Buch Shakespeare TODO einzulesen. Untersuchen Sie den Datensatz, indem Sie einen kurzen Auszug des Textes ausgeben.

(2) Erstellen Sie aus dem Text das Alphabet (`tokens`) und geben die Länge aus. Das Alphabet soll alle Zeichen des Textes enthalten.

* Hinweis: Sie können den `text` mit der Funktion `tokenize` in eine Liste umwandeln. Dieser kann dann zu einem [Set](https://docs.python.org/3/tutorial/datastructures.html#sets) gewandelt werden.

(3) Erstellen Sie für das Alphabet eine One Hot Kodierung und wenden diese auf den `text` an. Nutzen Sie dazu Ihre die Erkentnisse aus Aufgabe 7.1.

(4) Teilen Sie den kodierten Text in Sequenzen auf. Wie würden Sie die Länge der einzelnen Sequenzen wählen? 
Verwenden Sie dazu die Funktion vorgegebene Funktion `sequenze_split`.

* Erklärung: Wieso wird der Text in Sequenzen gesplittet?
</br>TODO Wieso machen wir das?

* Hinweis: Die Sequenzen sollten lang genug sein, dass das RNN Zusammenhänge in einem Satz (und ggf. darüber hinaus) erlenen kann. Die Sequenzlänge sollte kurz genug sein, damit eine ausreichende Anzahl für das Training vorhanden ist.

TODO RIOT Hinweis mit unserer Sequenzlänge.

(5) Erstellen Sie nun aus den sequenzierten Daten das Feature `X` und das Label `y`. Für die Vorhersage von einzelnen Zeichen ist jeweils das nächste Zeichen das Label des vorherigen Zeichens. Das Label `y` wird deshalb aus den gleichen Werte des Features `X` gebildet, ist jedoch um +1 verschoben. 


* Beispiel: Die Sequenz `Hello World!` kann in **X** `Hello World` mit dem zugehörigen Label **y** `ello World!` aufgeteilt werden.
* Hinweis: Untersuchen Sie die Rückgabe der Funktion `sequenze_split`. In welcher der drei Komponenten müssen Sie die Verschiebung vornehmen.

(6) Konvertieren Sie nun die Daten in ein Trainings- und Validierungsset. Nutzen Sie dafür die Funktion `train_test_split` (Eine Dokumentation finden Sie [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). 

In [4]:
# maximum text length 
# 1. Protection against "OutOfMemory" (poor RAM 😥)
# 2. Adjustment for making training faster (but you will gain less accuracy)
MAX_TEXT_LEN = 700_000 

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# TODO path_to_file = tf.keras.utils.get_file('german_news.txt', 'https://raw.githubusercontent.com/tblock/10kGNAD/master/articles.csv') # Taken from https://github.com/tblock/10kGNAD

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')
if len(text) > MAX_TEXT_LEN:
    print("Text is too long. Cutting it to", MAX_TEXT_LEN, f"characters. That is {100 * MAX_TEXT_LEN / len(text):6.2f} %")
    text = text[:MAX_TEXT_LEN]

# helper function for 7.2.4
def sequenze_split(coded_text, sequence_len):
    """
    Splits a given coded text (text converted with OneHot) into multiple subsequenzes
    """
    target_shape = (int(coded_text.shape[0] / (sequence_len + 1)) , (sequence_len + 1) , coded_text.shape[1])
    coded_text_seq = np.empty(target_shape, dtype=coded_text.dtype)
    for s in range(coded_text_seq.shape[0]):
        coded_text_seq[s] = coded_text[s * (sequence_len + 1):(s + 1) * (sequence_len + 1)]

    # returns 3D matrix (sequences, letters per sequence, letter as one hot coded vector)    
    return coded_text_seq    

# code here   
# Aufgabe 7.2.1 
print("Segment of text:")
print("-" * 60)
print(colored(text[:250], 'blue')) 
print("-" * 60)

# Aufgabe 7.2.2 - Alphabet erstellen
tokens = sorted(set(text))
print("Length of raw text:", len(text))
print("Numbers of tokens (unique letters): ", len(tokens))


# Aufgabe 7.2.3 - One Hot
one_hot = OneHot(tokens=tokens)
coded_text = one_hot.encode(text)
print("Shape of coded text: ", coded_text.shape)

# Aufgabe 7.2.4 - Aufteilung in Sequenzen
sequence_len = 150
coded_text_seq = sequenze_split(coded_text, sequence_len)
print("Shape of coded text split into sequences", coded_text_seq.shape)

# Aufgabe 7.2.5
X = coded_text_seq[:, :-1]
y = coded_text_seq[:, 1:]

# Aufgabe 7.2.6
validation_size = 1/4

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_size)

print("X_train:", X_train.shape, X_train.dtype)
print("y_train:", y_train.shape, y_train.dtype)
print("X_val  :", X_val.shape, X_val.dtype)
print("y_val  :", y_val.shape, y_val.dtype)

print("-" * 40)
print(colored(untokenize(one_hot.decode(X_train[0])), 'blue'))
print("-" * 40)
print(colored(untokenize(one_hot.decode(y_train[0])), 'green'))
print("-" * 40)    

Length of text: 1115394 characters
Text is too long. Cutting it to 700000 characters. That is  62.76 %
Segment of text:
------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

------------------------------------------------------------
Length of raw text: 700000
Numbers of tokens (unique letters):  65
Shape of coded text:  (700000, 66)
Shape of coded text split into sequences (6930, 101, 66)
X_train: (5197, 100, 66) bool
y_train: (5197, 100, 66) bool
X_val  : (1733, 100, 66) bool
y_val  : (1733, 100, 66) bool
----------------------------------------
 spend another such a night,
Though 'twere to buy a world of happy days,
So full of dismal terror wa
----------------------------------------
spend another such a night,
Though 'twere to 

🎉🎉🎉 **Geschafft!** 🎉🎉🎉

Sie haben nun aus einem Buch Trainingsdaten für ein RNN zur Vorhersage von Zeichen erstellt! 
- Text einlesen
- Text in Tokens umwandeln
- Alphabet festlegen
- One Hot Kodierung erstellen und anwenden
- In Sequenzen aufteilen
- `X` und `y` festlegen
- Aufteilung in Training und Validierungsset

... jetzt können Sie mit dem eigentlichen Modell fortfahren.

## 7.3 RNN Modell erstellen

In dieser Übung wird nun das rekurrente neuronale Netz zur Vorhersage von Buchstaben zu einer gegebenen Sequenz erstellt. Das Training wird in der nächsten Aufgabenteil durchgeführt.

In diesem Aufgabenteil wird ein Simple RNN (auch Vanilla bezeichnet) trainiert.

Sie verwenden dazu ein *SimpleRNN* (Link) 

Hier erstellen wir das eigentliche RNN. Hierfür nutzen wir ein Sequential Model von Keras mit einem RNN und einem Dense Layer mit Softmax zum erstellen der Ausgabe.

**Ihre Aufgaben**

(1) Bestimmen Sie nun die Parameter, die für das Modell benötigt werden.
- Shape der Eingabe/des Trainingdatensatzes. (Anzahl Zeichen, Länge eines vektorisierten Zeichens)
- Länge des Alphabets (inklusive des `unknown_tokens`). Also die Anzahl der Tokens, die das Modell vorhersagen können soll.
  

(2) Erstellen Sie das entsprechende [Sequential](https://keras.io/api/models/sequential/) Modell und fügen folgende Schichten hinzu:
- [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/) mit 1024 Units als Output Space (= Größe des Hidden State), dem Parameter `return_sequences=true` und der Shape der Eingabe. 
- [Dense](https://keras.io/api/layers/core_layers/dense/) mit der Anzahl der Tokens und der Aktivierungsfunktion `softmax`.

Kompilieren Sie anschließend das Modell mit der `categorical_crossentropy` Loss Function, dem `Adam` Optimizer und fügen als Metrics die `accuracy` hinzu.

In [5]:
# code here
# Aufgabe 7.3.1
# Length of the vocabulary in chars
vocab_size = X_train.shape[-1]
print("Numbers of unique tokens: ", vocab_size)

# The input shape
input_shape = X_train.shape[1:]
print("Input shape: ", input_shape)

# Number of RNN units
rnn_units = 2048

# Aufgabe 7.3.2 - SimpleRNN
model = tf.keras.Sequential(name="SimpleRNN_Model")
model.add(tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True, input_shape=input_shape, name="The_Brain"))
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax', name="The_Hand"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Zusatz
input_example_batch = X_train[0]
input_example_batch = input_example_batch.reshape((1, *input_example_batch.shape))

example_batch_predictions = model.predict(input_example_batch)
print(input_example_batch.shape, y_train[0].shape, example_batch_predictions.shape)

print("Test sequence:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(X_train[0])), 'blue'), "\n", "-"*40)
print("Expected result:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(y_train[0])), 'green'), "\n", "-"*40)
print("Untrained prediction:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(example_batch_predictions[0])), 'red'), "\n", "-"*40)

Numbers of unique tokens:  66
Input shape:  (100, 66)
Model: "SimpleRNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
The_Brain (SimpleRNN)        (None, 100, 2048)         4331520   
_________________________________________________________________
The_Hand (Dense)             (None, 100, 66)           135234    
Total params: 4,466,754
Trainable params: 4,466,754
Non-trainable params: 0
_________________________________________________________________
(1, 100, 66) (100, 66) (1, 100, 66)
Test sequence: 
 ---------------------------------------- 
  spend another such a night,
Though 'twere to buy a world of happy days,
So full of dismal terror wa 
 ----------------------------------------
Expected result: 
 ---------------------------------------- 
 spend another such a night,
Though 'twere to buy a world of happy days,
So full of dismal terror was 
 ----------------------------------------
Unt

## 7.4 RNN Modell trainieren


TODO 
Nun gehts ans trainieren. Nutzen Sie für den Anfang eine kleine Anzahl an epochen um herauszufinden wie viele Sie in einer ertragbaren Zeit machen können. Sollten Sie auf google Colab arbeiten könnte 20 hier ein guter Startwert sein

**Ihre Aufgaben**

(1) Nutze die Fit methode deines models um es auf den ertellten Trainingsdaten zu trainieren. Wenn Sie validierungsdaten erzeugt haben können Sie diese bei validation_data angeben. Um die Trainingszeit kurz zu halten kann es auch Sinn ergeben hier eine batch_size anzugeben

* Hinweis: Eine val_accuracy von circa 40% nach 10 Epochen oder von 50% nach 20 Epochen ist für dieses Modell natürlich noch lange nicht gut, aber ganz passabel. Erwarten Sie bitte nicht zu viel.

In [6]:
EPOCHS = 20
BATCH_SIZE = 32

In [ ]:
t1 = time.time()
hist = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  validation_data=(X_val, y_val))
t2 = time.time()

print("Training took: ", timedelta(seconds=t2-t1))

Epoch 1/20
163/163 [==============================] - 16s 95ms/step - loss: 3.6712 - accuracy: 0.1335 - val_loss: 3.2370 - val_accuracy: 0.1543
Epoch 2/20
163/163 [==============================] - 15s 90ms/step - loss: 3.1607 - accuracy: 0.1832 - val_loss: 2.8161 - val_accuracy: 0.2740
Epoch 3/20
163/163 [==============================] - 15s 91ms/step - loss: 2.9168 - accuracy: 0.2416 - val_loss: 2.7301 - val_accuracy: 0.2609
Epoch 4/20
163/163 [==============================] - 15s 91ms/step - loss: 2.7170 - accuracy: 0.2675 - val_loss: 2.5407 - val_accuracy: 0.2996
Epoch 5/20
163/163 [==============================] - 15s 92ms/step - loss: 2.5412 - accuracy: 0.2997 - val_loss: 2.5030 - val_accuracy: 0.3133
Epoch 6/20
163/163 [==============================] - 15s 91ms/step - loss: 2.4368 - accuracy: 0.3204 - val_loss: 2.3879 - val_accuracy: 0.3279
Epoch 7/20
163/163 [==============================] - 15s 90ms/step - loss: 2.4484 - accuracy: 0.3210 - val_loss: 2.3650 - val_accuracy:

(2) Plotten Sie accuracy und loss über die Epochen um bewerten zu können wie gut das Training lief

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(hist.history["loss"], label='loss')
ax.plot(hist.history["accuracy"], label='accuracy')
ax.plot(hist.history["val_loss"], label='validation loss')
ax.plot(hist.history["val_accuracy"], label='validation accuracy')
ax.set_xlabel("Epoch")
ax.legend()

fig.set_size_inches(20, 10)

plt.show()

In [ ]:
# Zusatz
input_example_batch = X_train[0]
input_example_batch = input_example_batch.reshape((1, *input_example_batch.shape))

example_batch_predictions = model.predict(input_example_batch)
print(input_example_batch.shape, y_train[0].shape, example_batch_predictions.shape)

print("Test sequence:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(X_train[0])), 'blue'), "\n", "-"*40)
print("Expected result:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(y_train[0])), 'green'), "\n", "-"*40)
print("Trained prediction:", "\n", "-" * 40, "\n", colored(untokenize(one_hot.decode(example_batch_predictions[0])), 'red'), "\n", "-"*40)

TODO Was hat er gelernt?

- Großschreibung nach Punkte
- Wörter
- ....

## x.5 Text prediction

(1) Schreiben Sie eine Methode, die für einen gegebenen Text das nächste Zeichen prognostiziert

In [ ]:
def predict(txt, one_hot, trainied_model):
    encoded_txt = one_hot.encode(tokenize(txt))
    prediction = trainied_model.predict(encoded_txt.reshape((1, *encoded_txt.shape)))
    return untokenize(one_hot.decode(prediction[0]))[-1]

(2) *Optional:* Nutzen Sie ipywidgets um live vorschläge zurück zu geben

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def verbose_prediction(txt, one_hot, trainied_model):
    if len(txt) == 0:
        txt = " "
    prediction = predict(txt=txt, one_hot=one_hot, trainied_model=trainied_model)
    print(f"I think the next will be \"{prediction}\" after you said \"{txt}\"")

w = widgets.interactive(verbose_prediction,
                        txt=widgets.Text(value='Hello Worl', placeholder='Type something', description='Your text:', disabled=False),
                        one_hot=widgets.fixed(one_hot),
                        trainied_model=widgets.fixed(model),
                       )

verbose_prediction("Spor", one_hot=one_hot, trainied_model=model)
verbose_prediction("Fußba", one_hot=one_hot, trainied_model=model)

display(w)

## x.6 *Optional* Vergleich mit LSTM und GRU und ...

Trainieren Sie nun modelle mit LSTM und GRU oder anderen Architekturen und vergleichen Sie lernperformance und Ergebnisse. Was fällt Ihnen auf?

In [ ]:
model_lstm = tf.keras.Sequential(name="LSTM_Model")
model_lstm.add(tf.keras.layers.LSTM(rnn_units, return_sequences=True, input_shape=input_shape, name="The_Brain"))
model_lstm.add(tf.keras.layers.Dense(vocab_size, activation='softmax', name="The_Hand"))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

model_gru = tf.keras.Sequential(name="GRU_Model")
model_gru.add(tf.keras.layers.LSTM(rnn_units, return_sequences=True, input_shape=input_shape, name="The_Brain"))
model_gru.add(tf.keras.layers.Dense(vocab_size, activation='softmax', name="The_Hand"))
model_gru.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.summary()


In [ ]:
t1 = time.time()
hist_lstm = model_lstm.fit(X_train, y_train,
                           batch_size=BATCH_SIZE,
                           epochs=EPOCHS,
                           validation_data=(X_val, y_val))
t2 = time.time()
t_lstm = t2 - t1

t1 = time.time()
hist_gru = model_gru.fit(X_train, y_train,
                         batch_size=BATCH_SIZE,
                         epochs=EPOCHS,
                         validation_data=(X_val, y_val))
t2 = time.time()
t_gru = t2 - t1

print("LSTM Training took:", timedelta(seconds=t_lstm))
print("GRU  Training took:", timedelta(seconds=t_gru))

In [ ]:
fig, ax = plt.subplots(1, 1)

for n, h in [("simple rnn", hist), ("lstm", hist_lstm), ("gru", hist_gru)]:

    ax.plot(h.history["loss"], label=f'{n} loss')
    ax.plot(h.history["accuracy"], label=f'{n} accuracy')
    ax.plot(h.history["val_loss"], label=f'{n} validation loss')
    ax.plot(h.history["val_accuracy"], label=f'{n} validation accuracy')

ax.set_xlabel("Epoch")
ax.legend()

fig.set_size_inches(20, 10)

plt.show()

## x.7 *Noch Optionaler* Ist das Netz jetzt ein Schriftsteller?

Was müsste man ändern um nun ein ganzes Buch erstellen zu lassen? Glauben Sie das Ergebniss wäre lesbar?

Hierfür wollen wir wenn möglich beliebig viele Zeichen generieren, dabei den hidden state behalten und nicht konstant das bisher geschriebene als Input wieder einfügen. Deshalb nutzen wir hier eine eigene Klasse für das Netz, die dann die Layer enthält, aber aich den Hidden state zurück geben kann

In [ ]:
class CustomRNNModel(tf.keras.Model):
    def __init__(self, vocab_size, rnn_units, rnn_layer_type = tf.keras.layers.GRU):
        super().__init__(self)
        # Put layers here. We will call them manually but store them here
        self.rnn_layer = rnn_layer_type(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax')

    def call(self, inputs, states=None, return_state=False, training=False):
        # Here we put our input through all out layers
        x = tf.cast(inputs, dtype=float)
        if states is None:
            # Get an initial kind of empty state
            states = self.rnn_layer.get_initial_state(x)
        
        x, states = self.rnn_layer(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
literatur_model = CustomRNNModel(vocab_size=vocab_size, rnn_units=rnn_units)

input_example_batch = X_train[0]
input_example_batch = input_example_batch.reshape((1, *input_example_batch.shape))

example_batch_predictions = literatur_model.predict(input_example_batch)
print(input_example_batch.shape, y_train[0].shape, example_batch_predictions.shape)

print("Test sequence:", "\n", "-" * 25, "\n", untokenize(one_hot.decode(X_train[0])), "\n", "-"*25)
print("Expected result:", "\n", "-" * 25, "\n", untokenize(one_hot.decode(y_train[0])), "\n", "-"*25)
print("Untrained prediction:", "\n", "-" * 25, "\n", untokenize(one_hot.decode(example_batch_predictions[0])), "\n", "-"*25)

literatur_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

literatur_model.summary()

In [ ]:
t1 = time.time()
hist_custom = literatur_model.fit(X_train, y_train,
                           batch_size=BATCH_SIZE,
                           epochs=EPOCHS,
                           validation_data=(X_val, y_val))
t2 = time.time()

print("Training took:", timedelta(seconds=t2-t1))

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(hist_custom.history["loss"], label='loss')
ax.plot(hist_custom.history["accuracy"], label='accuracy')
ax.plot(hist_custom.history["val_loss"], label='validation loss')
ax.plot(hist_custom.history["val_accuracy"], label='validation accuracy')
ax.set_xlabel("Epoch")
ax.legend()

fig.set_size_inches(20, 10)

plt.show()

In [ ]:
def create_text(amount_symbols, init_text, text_model):    
    # Tokenize text
    ret = tokenize(init_text)
    
    states = None
    current_char = one_hot.encode(ret, dtype=float)
    # For some reason we have to add one dimension
    current_char = current_char.reshape(1, *current_char.shape)
    
    for i in range(amount_symbols):
        current_char, states = text_model(current_char, return_state = True, states = states)
        # Only add last char if several got 
        ret.append(one_hot.decode(current_char[0])[-1])
        
    return untokenize(ret)
    

In [ ]:
created_text = create_text(250, "Sport", literatur_model)

print(created_text)